In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Read the files
df_final_demo = pd.read_csv('df_final_demo.txt', delimiter=',')
df_final_experiment_clients = pd.read_csv('df_final_experiment_clients.txt', delimiter=',')
df_final_web_data_pt_1 = pd.read_csv('df_final_web_data_pt_1.txt', delimiter=',')
df_final_web_data_pt_2 = pd.read_csv('df_final_web_data_pt_2.txt', delimiter=',')

In [ ]:
# Check for missing values
print(df_final_demo.isnull().sum())

# Fill missing values or drop rows with significant missing data
df_final_demo['clnt_age'] = df_final_demo['clnt_age'].fillna(df_final_demo['clnt_age'].median())
df_final_demo['gendr'] = df_final_demo['gendr'].fillna('Unknown')

# Replace negative or invalid values with NaN and handle them
df_final_demo['bal'] = df_final_demo['bal'].apply(lambda x: np.nan if x < 0 else x)
df_final_demo = df_final_demo.dropna()  # Drop rows with invalid values

In [ ]:
#test case
print(df_final_experiment_clients.head())

In [ ]:
#clean df_final_experiment
if 'Variation' in df_final_experiment_clients.columns:
    print(df_final_experiment_clients['Variation'].unique())
else:
    print("'Variation' column not found. Cannot proceed with analysis.")


In [ ]:
#df_final_experiment_clients = pd.read_csv('df_final_experiment_clients.csv', delimiter=',')

# Ensure Variation contains only valid entries
print(df_final_experiment_clients['Variation'].unique())

# Fix invalid variations (if any)
valid_variations = ['Test', 'Control']
df_final_experiment_clients = df_final_experiment_clients[
    df_final_experiment_clients['Variation'].isin(valid_variations)
]

In [ ]:
# Concatenate the two parts
df_final_web_data = pd.concat([df_final_web_data_pt_1, df_final_web_data_pt_2], ignore_index=True)

# Convert `date_time` to datetime format
df_final_web_data['date_time'] = pd.to_datetime(df_final_web_data['date_time'])

# Check for duplicates and drop them
df_final_web_data = df_final_web_data.drop_duplicates()

# Ensure `process_step` is valid (e.g., integers or known steps)
print(df_final_web_data['process_step'].unique())


In [ ]:
merged_data = pd.merge(df_final_demo, df_final_experiment_clients, on='client_id', how='inner')

# Merge the web data
merged_data = pd.merge(merged_data, df_final_web_data, on='client_id', how='inner')

In [ ]:
merged_data.to_csv('cleaned_data.csv', index=False)


In [ ]:
#print out csv header 
df_all_clean = pd.read_csv("cleaned_data.csv", dtype={'Variation': 'str'}, low_memory=False)
print(df_all_clean.dtypes)

In [ ]:
# Check the structure of the data
print(df_all_clean.info())
print(df_all_clean.head())

In [ ]:
print(df_all_clean.describe())
print(df_all_clean['gendr'].value_counts())

In [ ]:
#Age distribution visualization

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Set pastel color for axis labels
pastel_color = '#AEC6CF'  # Example pastel blue from a pastel palette

# Plot with pastel style
merged_data['clnt_age'].hist(bins=20, color='#5F9EA0', edgecolor='white')  # Add pastel orange for the bars
plt.title('Age Distribution', color='#00CED1')  # Pastel green title
plt.xlabel('Age', color='#008B8B')  # Use pastel blue for the label
plt.ylabel('Frequency', color='#20B2AA')  # Use pastel blue for the label
plt.show()

In [ ]:
merged_data.plot.scatter(x='num_accts // Number of Accounts', y='bal // Balance', alpha=0.5)
plt.title('Balance vs. Number of Accounts')
plt.show()


In [ ]:
print(merged_data.groupby('Variation')['clnt_age'].mean())
print(merged_data.groupby('Variation')['bal'].mean())


In [ ]:
# Filter for the 'confirm' step
completion_data = merged_data[merged_data['process_step'] == 'confirm']

# Calculate completion rates
total_test = len(merged_data[merged_data['Variation'] == 'Test'])
total_control = len(merged_data[merged_data['Variation'] == 'Control'])
confirm_test = len(completion_data[completion_data['Variation'] == 'Test'])
confirm_control = len(completion_data[completion_data['Variation'] == 'Control'])

test_completion_rate = confirm_test / total_test
control_completion_rate = confirm_control / total_control

print(f"Test Completion Rate: {test_completion_rate}")
print(f"Control Completion Rate: {control_completion_rate}")


In [ ]:
merged_data['date_time'] = pd.to_datetime(merged_data['date_time'])

# Sort data by client and timestamp
merged_data = merged_data.sort_values(by=['client_id', 'date_time'])

# Calculate time differences
merged_data['time_diff'] = merged_data.groupby('client_id')['date_time'].diff()

# Average time spent per step
time_per_step = merged_data.groupby('process_step')['time_diff'].mean()
print(time_per_step)


In [ ]:
# Calculate error rates with division by zero handling
test_error_rate = error_test / total_steps_test if total_steps_test > 0 else 0
control_error_rate = error_control / total_steps_control if total_steps_control > 0 else 0

print(f"Test Error Rate: {test_error_rate}")
print(f"Control Error Rate: {control_error_rate}")


In [ ]:
# Ensure process_step is numeric
merged_data['process_step'] = pd.to_numeric(merged_data['process_step'], errors='coerce')

# Drop rows with NaN in process_step
merged_data = merged_data.dropna(subset=['process_step'])

# Calculate step differences
merged_data['step_diff'] = merged_data.groupby('client_id')['process_step'].diff()

# Identify backward steps
backward_steps = merged_data[merged_data['step_diff'] < 0]

# Calculate error rate
total_steps_test = len(merged_data[merged_data['Variation'] == 'Test'])
total_steps_control = len(merged_data[merged_data['Variation'] == 'Control'])
error_test = len(backward_steps[backward_steps['Variation'] == 'Test'])
error_control = len(backward_steps[backward_steps['Variation'] == 'Control'])

test_error_rate = error_test / total_steps_test
control_error_rate = error_control / total_steps_control

print(f"Test Error Rate: {test_error_rate}")
print(f"Control Error Rate: {control_error_rate}")
